In [ ]:
%%time
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import math
import xgboost as xgb
from scipy.stats import skew
from scipy import stats
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, auc, classification_report, roc_curve, roc_auc_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from datetime import datetime

CPU times: user 92 µs, sys: 10 µs, total: 102 µs
Wall time: 105 µs


In [ ]:
import re
import string

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# from bs4 import BeautifulSoup
%matplotlib inline

import pickle as pkl

In [ ]:
df = pd.read_csv("/ks-projects-201801.csv.zip")

In [ ]:
df

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378656,999976400,ChknTruk Nationwide Charity Drive 2014 (Canceled),Documentary,Film & Video,USD,2014-10-17,50000.0,2014-09-17 02:35:30,25.0,canceled,1,US,25.0,25.0,50000.00
378657,999977640,The Tribe,Narrative Film,Film & Video,USD,2011-07-19,1500.0,2011-06-22 03:35:14,155.0,failed,5,US,155.0,155.0,1500.00
378658,999986353,Walls of Remedy- New lesbian Romantic Comedy f...,Narrative Film,Film & Video,USD,2010-08-16,15000.0,2010-07-01 19:40:30,20.0,failed,1,US,20.0,20.0,15000.00
378659,999987933,BioDefense Education Kit,Technology,Technology,USD,2016-02-13,15000.0,2016-01-13 18:13:53,200.0,failed,6,US,200.0,200.0,15000.00


In [ ]:
df.nunique()

ID                  378661
name                375764
category               159
main_category           15
currency                14
deadline              3164
goal                  8353
launched            378089
pledged              62130
state                    6
backers               3963
country                 23
usd pledged          95455
usd_pledged_real    106065
usd_goal_real        50339
dtype: int64

In [ ]:
df.isnull().sum()

ID                     0
name                   4
category               0
main_category          0
currency               0
deadline               0
goal                   0
launched               0
pledged                0
state                  0
backers                0
country                0
usd pledged         3797
usd_pledged_real       0
usd_goal_real          0
dtype: int64

In [ ]:
df['log_goal'] = np.log10(df['goal'])

In [ ]:
df.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,log_goal
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95,3.000000
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00,4.477121
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00,4.653213
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00,3.698970
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00,4.290035


In [ ]:
df['launched'] = pd.to_datetime(df['launched'])
df['deadline'] = pd.to_datetime(df['deadline'])

In [ ]:
df.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,log_goal
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95,3.000000
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00,4.477121
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00,4.653213
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00,3.698970
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00,4.290035


In [ ]:
df['duration'] = df['deadline'] - df['launched']

In [ ]:
df.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,log_goal,duration
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95,3.000000,58 days 11:47:32
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00,4.477121,59 days 19:16:03
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00,4.653213,44 days 23:39:10
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00,3.698970,29 days 20:35:49
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00,4.290035,55 days 15:24:57


In [ ]:
df['state'].unique()

array(['failed', 'canceled', 'successful', 'live', 'undefined',
       'suspended'], dtype=object)

In [ ]:
df.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,log_goal,duration
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95,3.000000,58 days 11:47:32
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00,4.477121,59 days 19:16:03
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00,4.653213,44 days 23:39:10
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00,3.698970,29 days 20:35:49
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00,4.290035,55 days 15:24:57


In [ ]:
df['state'].value_counts()

failed        197719
successful    133956
canceled       38779
undefined       3562
live            2799
suspended       1846
Name: state, dtype: int64

In [ ]:
df.drop(df[(df['state'] == 'undefined')].index,inplace=True)

In [ ]:
df.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,log_goal,duration
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95,3.000000,58 days 11:47:32
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00,4.477121,59 days 19:16:03
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00,4.653213,44 days 23:39:10
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00,3.698970,29 days 20:35:49
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00,4.290035,55 days 15:24:57


In [ ]:
df['state'].value_counts()

failed        197719
successful    133956
canceled       38779
live            2799
suspended       1846
Name: state, dtype: int64

In [ ]:
df["state"].replace({"failed": "Failed", "successful": "Success","canceled": "Failed","live": "Success","suspended": "Failed"}, inplace=True)

In [ ]:
df.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,log_goal,duration
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,Failed,0,GB,0.0,0.0,1533.95,3.000000,58 days 11:47:32
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,Failed,15,US,100.0,2421.0,30000.00,4.477121,59 days 19:16:03
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,Failed,3,US,220.0,220.0,45000.00,4.653213,44 days 23:39:10
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,Failed,1,US,1.0,1.0,5000.00,3.698970,29 days 20:35:49
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,Failed,14,US,1283.0,1283.0,19500.00,4.290035,55 days 15:24:57


In [ ]:
df['state'].value_counts(normalize=True)*100

Failed     63.541625
Success    36.458375
Name: state, dtype: float64

In [ ]:
my_features = ['country', 'currency', 'duration', 'goal']

In [ ]:
dtrain = df[my_features]
dtest = df[my_features]

In [ ]:
y = df.state

X_train, X_val, y_train, y_val = train_test_split(df, y, test_size=0.1, random_state=0)
categorical_cols = [cname for cname in dtrain.columns if
                    dtrain[cname].nunique() < 10 and 
                    dtrain[cname].dtype == "object"]

numerical_cols = [cname for cname in dtrain.columns if 
                dtrain[cname].dtype in ['int64', 'float64']]

In [ ]:
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(337589, 17)
(37510, 17)
(337589,)
(37510,)


In [ ]:
numerical_transformer = SimpleImputer(strategy='mean')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

sc = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [ ]:
Results = pd.DataFrame({'Model': [], 'Accuracy Score': []})

In [ ]:
knnc = KNeighborsClassifier()

knn = Pipeline(steps=[('preprocessor', preprocessor),
                      ('sc',StandardScaler()),
                      ('model', knnc)
])

knn.fit(X_train, y_train)

predsKNN = knn.predict(X_val)


print('Accuracy:', accuracy_score(y_val, predsKNN))
print('CR:', classification_report(y_val, predsKNN))
print('CM:',confusion_matrix(y_val, predsKNN))

res = pd.DataFrame({'Model': ['KNN'],
                    'Accuracy Score': [accuracy_score(y_val, predsKNN)]})
Results = Results.append(res)

Accuracy: 0.6095707811250334
CR:               precision    recall  f1-score   support

      Failed       0.66      0.78      0.72     23805
     Success       0.45      0.32      0.37     13705

    accuracy                           0.61     37510
   macro avg       0.56      0.55      0.54     37510
weighted avg       0.59      0.61      0.59     37510

CM: [[18527  5278]
 [ 9367  4338]]


In [ ]:
print('Accuracy:', accuracy_score(y_val, predsKNN))
print('CR:', classification_report(y_val, predsKNN))
print('CM:',confusion_matrix(y_val, predsKNN))

res = pd.DataFrame({'Model': ['KNN'],
                    'Accuracy Score': [accuracy_score(y_val, predsKNN)]})
Results = Results.append(res)

Accuracy: 0.6095707811250334
CR:               precision    recall  f1-score   support

      Failed       0.66      0.78      0.72     23805
     Success       0.45      0.32      0.37     13705

    accuracy                           0.61     37510
   macro avg       0.56      0.55      0.54     37510
weighted avg       0.59      0.61      0.59     37510

CM: [[18527  5278]
 [ 9367  4338]]


In [ ]:
from xgboost import XGBClassifier
model_xgb = XGBClassifier(n_estimators= 200, learning_rate = 0.1, max_depth=2, use_label_encoder=False)

xgb = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model_xgb', model_xgb)
])


xgb.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['goal']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  [])])),
                ('model_xgb',
                 XGBClassifier(max_depth=2, n_estimators=200,
                               use_label_encoder=False))])

In [ ]:
with open("model_k.pkl","wb") as f:
    pkl.dump(model_xgb, f)

In [ ]:
with open("model_k.pkl", 'rb') as f:
    pkl.load(f)

In [ ]:
predictions = xgb.predict(X_val)
print('Accuracy:', accuracy_score(y_val, predictions))
print('CR:', classification_report(y_val, predictions))
print('CM:',confusion_matrix(y_val, predictions))

res = pd.DataFrame({'Model': ['XGB'],
                    'Accuracy Score': [accuracy_score(y_val, predictions)]})
Results = Results.append(res)

Accuracy: 0.6380965075979739
CR:               precision    recall  f1-score   support

      Failed       0.65      0.94      0.77     23805
     Success       0.52      0.12      0.19     13705

    accuracy                           0.64     37510
   macro avg       0.58      0.53      0.48     37510
weighted avg       0.60      0.64      0.56     37510

CM: [[22305  1500]
 [12075  1630]]


In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=0,criterion='entropy')


clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('sc',StandardScaler()),
                      ('model', model)
])


clf.fit(X_train, y_train)


preds = clf.predict(X_val)
print('Accuracy:', accuracy_score(y_val, preds))
print('CR:', classification_report(y_val, preds))
print('CM:',confusion_matrix(y_val, preds))

res = pd.DataFrame({'Model': ['RandomForest'],
                    'Accuracy Score': [accuracy_score(y_val, preds)]})
Results = Results.append(res)

Accuracy: 0.6354572114102905
CR:               precision    recall  f1-score   support

      Failed       0.65      0.91      0.76     23805
     Success       0.50      0.16      0.24     13705

    accuracy                           0.64     37510
   macro avg       0.58      0.53      0.50     37510
weighted avg       0.60      0.64      0.57     37510

CM: [[21695  2110]
 [11564  2141]]
